In [1]:
%cd ../src

/home/ubuntu/SPVD_Lightning/src


In [2]:
import torch
torch.set_float32_matmul_precision('medium')

In [3]:
from datasets.modelnet40.modelnet40_loader import get_dataloaders, ModelNet40

path = "../data/ModelNet40"
categories = ['bottle', 'car']

tr, te = get_dataloaders(path, categories=categories)

Loading renders for bottle: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:14<00:00,  6.88it/s]


In [4]:
from models import *

m = SPVD_S()

lr = 1e-4
model = DiffusionBase(m, lr=lr)

/opt/conda/envs/spvd/lib/python3.10/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


In [5]:
import lightning as L
from lightning.pytorch.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

logger = TensorBoardLogger("../logs/ModelNet40/bottle-car", name="test")

checkpoint_name = "-".join(categories) if categories else "all"
epochs = 100
checkpoint_callback = ModelCheckpoint(
    dirpath=f'../checkpoints/ModelNet40/{checkpoint_name}',
    filename="{epoch:02d}-{val_loss:.2f}", 
    every_n_epochs=5
)

trainer = L.Trainer(
    logger=logger,
    log_every_n_steps=5,
    max_epochs=epochs, 
    gradient_clip_val=10.0, 
    callbacks=[checkpoint_callback]
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(model=model, train_dataloaders=tr, val_dataloaders=te)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type    | Params | Mode 
------------------------------------------
0 | model | SPVUnet | 23.1 M | train
------------------------------------------
23.1 M    Trainable params
0         Non-trainable params
23.1 M    Total params
92.362    Total estimated model params size (MB)
229       Modules in train mode
0         Modules in eval mode


Sanity Checking: |                                                                                            …

In [ ]:
from utils.schedulers import DDPMSparseSchedulerGPU

ddpm_sched = DDPMSparseSchedulerGPU(n_steps=1000, beta_min=0.0001, beta_max=0.02, pres=1e-5)

preds = ddpm_sched.sample(model.cuda(), 16, 2048)

from utils.visualization import visualize_notebook
visualize_notebook(preds, x_offset=2.5, y_offset=2.5, point_size=0.025)